In [1]:
from groq import Groq
import os
import sys
import anthropic
import ollama
import random
import pandas as pd
from tqdm import tqdm
from google.generativeai.types import RequestOptions
from google.api_core import retry
from typing import List, Tuple
import json
from openai import OpenAI
import datetime
import openai
import time

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)

if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from concurrent.futures import ThreadPoolExecutor, TimeoutError

# prompts

### Zero Shot - Vanilla CoT

In [2]:
zero_shot_vanilla_cot = """
Think through your answer step by step. Put the concise form of your final answer in curly brackets e.g. {A}, {True} or {3.0}.
"""

In [3]:
def get_prompt(prompt_type: str, few_shot_prompt: str, question: str) -> str:
    prompts = {
        "zero_shot_vanilla_cot": f"{question}\n{zero_shot_vanilla_cot}",
    }
    return prompts[prompt_type]

def save_results(save_path: str, ids: List[str], questions: List[str], answers: List[str], append: bool = False):
    df = pd.DataFrame({'id': ids, 'question': questions, 'answer': answers})
    if append and os.path.exists(save_path):
        df.to_csv(save_path, mode='a', index=False, header=False)
    else:
        df.to_csv(save_path, index=False)

def read_jsonl_file(filepath: str) -> List[dict]:
    data = []
    with open(filepath, 'r') as file:
        for line in file:
            json_obj = json.loads(line)
            data.append(json_obj)
    return data

def load_data_size_specific(data_path: str, sample_size: int = 0, random_seed: int = 0):
    random.seed(random_seed)

    if data_path.endswith('.jsonl'):
        data = read_jsonl_file(data_path)
    elif data_path.endswith('.json'):
        with open(data_path, 'r') as file:
            data = json.load(file)
    
    question_length = 0
    eligible_data = [x for x in data if len(x["question"]) >= question_length]
    
    if sample_size > 0 and sample_size < len(eligible_data):
        sampled_data = random.sample(eligible_data, sample_size)
    else:
        sampled_data = eligible_data
    
    ids = [x["id"] for x in sampled_data]
    questions = [x["question"] for x in sampled_data]
    
    return ids, questions

def load_already_answered_ids(save_path: str) -> set:
    if os.path.exists(save_path):
        df = pd.read_csv(save_path)
        answered_ids = set(df['id'].tolist())
        # print(f"Loaded {len(answered_ids)} already answered IDs from: {save_path}")
        print(f"Already answered IDs: {answered_ids}")
        return answered_ids
    else:
        print(f"No existing save file found at: {save_path}. Starting fresh.")
        return set()

def initialize_save_file(save_path: str):
    if not os.path.exists(save_path):
        # Create an empty DataFrame with headers and save
        df = pd.DataFrame(columns=['id', 'question', 'answer'])
        df.to_csv(save_path, index=False)
        print(f"Initialized new save file with headers at: {save_path}")

In [20]:
def query_4o(prompt: str) -> str:
    client = OpenAI()

    completion = client.chat.completions.create(
        model="gpt-4o-2024-08-06",
        messages=[
            {
                "role": "user",
                "content": f"{prompt}"
            }
        ],
        temperature=0
    )

    return completion.choices[0].message.content

def query_llama(prompt):
    client = openai.OpenAI(
        api_key=os.environ.get("SAMBANOVA_API_KEY"),
        base_url="https://api.sambanova.ai/v1",
    )

    response = client.chat.completions.create(
        model='Meta-Llama-3.1-8B-Instruct',
            messages=[
                {
                    "role": "user",
                    "content": prompt,
                }
            ],
        temperature=0.6, # Meta default
        top_p = 0.9 # Meta default
    )
    time.sleep(2)  # Pause execution for 2 seconds
    return response.choices[0].message.content

def query_llama_70b(prompt):
    client = openai.OpenAI(
        api_key=os.environ.get("SAMBANOVA_API_KEY"),
        base_url="https://api.sambanova.ai/v1",
    )

    response = client.chat.completions.create(
        model='Meta-Llama-3.1-70B-Instruct',
            messages=[
                {
                    "role": "user",
                    "content": prompt,
                }
            ],
        temperature=0.6, # Meta default
        top_p = 0.9 # Meta default
    )
    time.sleep(2)  # Pause execution for 2 seconds
    return response.choices[0].message.content

def query_llama_405b(prompt):
    client = openai.OpenAI(
        api_key=os.environ.get("SAMBANOVA_API_KEY"),
        base_url="https://api.sambanova.ai/v1",
    )

    response = client.chat.completions.create(
        model='Meta-Llama-3.1-405B-Instruct',
            messages=[
                {
                    "role": "user",
                    "content": prompt,
                }
            ],
        temperature=0.6, # Meta default
        top_p = 0.9 # Meta default
    )
    time.sleep(5)  # Pause execution for 2 seconds
    return response.choices[0].message.content

In [17]:
def query_llm(llm_model: str, ids: List[str], questions: List[str], few_shot_prompt: str, prompt_type: str, save_path: str, already_answered_ids: set) -> Tuple[List[str], List[str], List[str]]:
    answers = []
    ids_can_be_answered = []
    questions_can_be_answered = []
    
    for id, q in tqdm(zip(ids, questions), total=len(ids)):
        # print(q)
        # print(f"Processing ID: {id}")
        if id in already_answered_ids:
            print(f"Skipping: {id}", end=' ')
            continue
        if id == 1146: # weird ID that breaks llama
            continue
        
        prompt = get_prompt(prompt_type, few_shot_prompt, q)
        try:
            if llm_model == 'gemini':
                answer = query_gemini(prompt, id)
            elif llm_model == 'claude':
                answer = query_claude(prompt)
            elif llm_model == '4o':
                # answer = query_4o_multiturn(prompt)
                if prompt_type == 'multi_convo':
                    fact_prompt = get_prompt(prompt_type="fact_prompt", few_shot_prompt="", question=q)
                    
                    answer_prompt = get_prompt(prompt_type="answer_prompt_data", few_shot_prompt="", question=q)
                    answer = query_4o_multiconvo(fact_prompt=fact_prompt, answer_prompt=answer_prompt, extracted_question=q)
                else:
                    answer = query_4o(prompt)
                
            elif llm_model == 'llama3.18b':
                answer = query_llama(prompt)
            elif llm_model == 'llama3.170b':
                answer = query_llama_70b(prompt)
            elif llm_model == 'llama3.1405b':
                answer = query_llama_405b(prompt)
            else:
                raise ValueError(f"Unsupported LLM model: {llm_model}")
            # print(f"Answer for ID {id}: {answer}")
            
            answers.append(answer)
            questions_can_be_answered.append(q)
            ids_can_be_answered.append(id)

            # Save after each answer
            save_results(save_path, [id], [q], [answer], append=True)
        except Exception as e:
            print(f"Error processing question {id}: {str(e)}")
            continue
    
    return ids_can_be_answered, questions_can_be_answered, answers

# Driver

In [13]:
json_datasets = ['logical_deduction_seven_objects','reasoning_about_colored_objects']
jsonl_datasets = ['GSM8K', 'date', 'GSM_Plus', 'MultiArith', 'ASDiv', 'SVAMP', 'AQUA', 'p_GSM8K', 'StrategyQA', 'commonsenseQA','SPARTQA']
all_datasets = jsonl_datasets + json_datasets

In [21]:
def run_model(llm_model, prompt_type, few_shot_txt, sample_size, project_root):
    for dataset in all_datasets:
        print(f"------Processing dataset: {dataset}-------")
        if few_shot_txt:
            fewshot_prompt_path = os.path.join(project_root, "prompt", dataset, few_shot_txt)
            
        save_dir = os.path.join(project_root, 'logan/results/final/VanillaCoT', dataset, f'{llm_model}')
        os.makedirs(save_dir, exist_ok=True)  # Ensure the directory exists
        save_path = os.path.join(save_dir, f'{prompt_type}_{few_shot_txt}_{dataset}_{llm_model}.csv')

        if dataset in json_datasets:
            data_path = os.path.join(project_root, 'data', dataset, 'test.json')
        else:
            data_path = os.path.join(project_root, 'data', dataset, 'test.jsonl')


        ids, questions = load_data_size_specific(data_path, sample_size=sample_size)
        if few_shot_txt:
            with open(fewshot_prompt_path, 'r') as file:
                few_shot_prompt = file.read()
        else:
            few_shot_prompt = ""

        initialize_save_file(save_path)
        already_answered_ids = load_already_answered_ids(save_path)

        ids_answered, questions_answered, answers = query_llm(
            llm_model=llm_model,
            ids=ids,
            questions=questions,
            few_shot_prompt=few_shot_prompt,
            prompt_type=prompt_type,
            save_path=save_path,
            already_answered_ids=already_answered_ids
        )

        print(f"Processing complete for {dataset}. {len(ids_answered)} new answers saved to {save_path}.")

In [24]:
llm_model = 'llama3.1405b'
project_root = '/Users/log/Github/textual_grounding/'
prompt_type = 'zero_shot_vanilla_cot'
few_shot_txt = None
sample_size = 400

run_model(llm_model, prompt_type, few_shot_txt, sample_size, project_root)

------Processing dataset: GSM8K-------
Already answered IDs: {0, 2, 3, 4, 6, 8, 18, 25, 27, 28, 29, 30, 33, 36, 38, 42, 44, 46, 49, 54, 62, 64, 65, 67, 72, 74, 79, 80, 82, 83, 85, 86, 89, 92, 93, 95, 100, 101, 102, 111, 117, 118, 125, 127, 128, 130, 132, 141, 143, 147, 151, 153, 159, 161, 164, 165, 166, 167, 168, 170, 183, 186, 187, 191, 194, 196, 200, 202, 205, 206, 207, 222, 223, 225, 226, 235, 236, 237, 239, 240, 243, 245, 253, 254, 255, 266, 267, 271, 273, 276, 283, 285, 286, 291, 296, 299, 300, 306, 309, 314, 315, 316, 322, 324, 327, 331, 336, 339, 343, 345, 346, 349, 351, 358, 365, 367, 369, 373, 376, 377, 378, 382, 384, 385, 386, 387, 389, 390, 391, 393, 394, 398, 403, 404, 406, 408, 412, 416, 418, 421, 424, 425, 431, 433, 436, 437, 440, 441, 447, 448, 451, 452, 454, 456, 459, 482, 485, 488, 489, 490, 491, 492, 495, 497, 499, 500, 509, 513, 525, 530, 531, 532, 533, 535, 555, 561, 562, 563, 564, 569, 570, 573, 574, 575, 577, 578, 589, 594, 596, 597, 598, 599, 616, 617, 619, 621, 

100%|██████████| 400/400 [00:00<00:00, 650027.74it/s]

Skipping: 788 Skipping: 861 Skipping: 82 Skipping: 530 Skipping: 1047 Skipping: 995 Skipping: 829 Skipping: 621 Skipping: 976 Skipping: 733 Skipping: 1194 Skipping: 447 Skipping: 1033 Skipping: 285 Skipping: 577 Skipping: 286 Skipping: 194 Skipping: 1266 Skipping: 513 Skipping: 1090 Skipping: 1232 Skipping: 300 Skipping: 635 Skipping: 202 Skipping: 151 Skipping: 676 Skipping: 966 Skipping: 206 Skipping: 724 Skipping: 889 Skipping: 647 Skipping: 1251 Skipping: 418 Skipping: 1131 Skipping: 1310 Skipping: 906 Skipping: 1067 Skipping: 533 Skipping: 127 Skipping: 1123 Skipping: 28 Skipping: 191 Skipping: 816 Skipping: 2 Skipping: 1253 Skipping: 1010 Skipping: 682 Skipping: 499 Skipping: 666 Skipping: 128 Skipping: 391 Skipping: 1162 Skipping: 454 Skipping: 488 Skipping: 291 Skipping: 1112 Skipping: 917 Skipping: 186 Skipping: 164 Skipping: 655 Skipping: 1040 Skipping: 1002 Skipping: 223 Skipping: 617 Skipping: 1128 Skipping: 596 Skipping: 255 Skipping: 1121 Skipping: 681 Skipping: 1106 Skip

100%|██████████| 359/359 [00:00<00:00, 702638.89it/s]


Skipping: 0 Skipping: 1 Skipping: 2 Skipping: 3 Skipping: 4 Skipping: 5 Skipping: 6 Skipping: 7 Skipping: 8 Skipping: 9 Skipping: 10 Skipping: 11 Skipping: 12 Skipping: 13 Skipping: 14 Skipping: 15 Skipping: 16 Skipping: 17 Skipping: 18 Skipping: 19 Skipping: 20 Skipping: 21 Skipping: 22 Skipping: 23 Skipping: 24 Skipping: 25 Skipping: 26 Skipping: 27 Skipping: 28 Skipping: 29 Skipping: 30 Skipping: 31 Skipping: 32 Skipping: 33 Skipping: 34 Skipping: 35 Skipping: 36 Skipping: 37 Skipping: 38 Skipping: 39 Skipping: 40 Skipping: 41 Skipping: 42 Skipping: 43 Skipping: 44 Skipping: 45 Skipping: 46 Skipping: 47 Skipping: 48 Skipping: 49 Skipping: 50 Skipping: 51 Skipping: 52 Skipping: 53 Skipping: 54 Skipping: 55 Skipping: 56 Skipping: 57 Skipping: 58 Skipping: 59 Skipping: 60 Skipping: 61 Skipping: 62 Skipping: 63 Skipping: 64 Skipping: 65 Skipping: 66 Skipping: 67 Skipping: 68 Skipping: 69 Skipping: 70 Skipping: 71 Skipping: 72 Skipping: 73 Skipping: 74 Skipping: 75 Skipping: 76 Skipping:

...

Skipping: 3458 Skipping: 1577 Skipping: 3104 Skipping: 1722 Skipping: 165 Skipping: 1060 Skipping: 2094 Skipping: 1990 Skipping: 1658 Skipping: 3210 Skipping: 3399 Skipping: 1242 Skipping: 1952 Skipping: 1466 Skipping: 2389 Skipping: 894 Skipping: 2067 Skipping: 570 Skipping: 1154 Skipping: 572 Skipping: 3095 Skipping: 388 Skipping: 2532 Skipping: 3274 Skipping: 1026 Skipping: 2181 Skipping: 2888 Skipping: 3318 Skipping: 2465 Skipping: 601 Skipping: 1270 Skipping: 404 Skipping: 2989 Skipping: 302 Skipping: 3483 Skipping: 2801 Skipping: 1352 Skipping: 1933 Skipping: 2292 Skipping: 412 Skipping: 1449 Skipping: 1778 Skipping: 1295 Skipping: 2502 Skipping: 2623 Skipping: 837 Skipping: 2263 Skipping: 1953 Skipping: 1813 Skipping: 3544 Skipping: 2135 Skipping: 1066 Skipping: 255 Skipping: 3297 Skipping: 2247 Skipping: 57 Skipping: 382 Skipping: 2947 Skipping: 3441 Skipping: 1633 Skipping: 2909 Skipping: 3378 Skipping: 3215 Skipping: 2736 Skipping: 2561 Skipping: 4 Skipping: 2506 Skipping: 20

100%|██████████| 250/250 [00:00<00:00, 972705.01it/s]


Skipping: 0 Skipping: 1 Skipping: 2 Skipping: 3 Skipping: 4 Skipping: 5 Skipping: 6 Skipping: 7 Skipping: 8 Skipping: 9 Skipping: 10 Skipping: 11 Skipping: 12 Skipping: 13 Skipping: 14 Skipping: 15 Skipping: 16 Skipping: 17 Skipping: 18 Skipping: 19 Skipping: 20 Skipping: 21 Skipping: 22 Skipping: 23 Skipping: 24 Skipping: 25 Skipping: 26 Skipping: 27 Skipping: 28 Skipping: 29 Skipping: 30 Skipping: 31 Skipping: 32 Skipping: 33 Skipping: 34 Skipping: 35 Skipping: 36 Skipping: 37 Skipping: 38 Skipping: 39 Skipping: 40 Skipping: 41 Skipping: 42 Skipping: 43 Skipping: 44 Skipping: 45 Skipping: 46 Skipping: 47 Skipping: 48 Skipping: 49 Skipping: 50 Skipping: 51 Skipping: 52 Skipping: 53 Skipping: 54 Skipping: 55 Skipping: 56 Skipping: 57 Skipping: 58 Skipping: 59 Skipping: 60 Skipping: 61 Skipping: 62 Skipping: 63 Skipping: 64 Skipping: 65 Skipping: 66 Skipping: 67 Skipping: 68 Skipping: 69 Skipping: 70 Skipping: 71 Skipping: 72 Skipping: 73 Skipping: 74 Skipping: 75 Skipping: 76 Skipping:

100%|██████████| 250/250 [00:00<00:00, 648069.22it/s]

Skipping: 0 Skipping: 1 Skipping: 2 Skipping: 3 Skipping: 4 Skipping: 5 Skipping: 6 Skipping: 7 Skipping: 8 Skipping: 9 Skipping: 10 Skipping: 11 Skipping: 12 Skipping: 13 Skipping: 14 Skipping: 15 Skipping: 16 Skipping: 17 Skipping: 18 Skipping: 19 Skipping: 20 Skipping: 21 Skipping: 22 Skipping: 23 Skipping: 24 Skipping: 25 Skipping: 26 Skipping: 27 Skipping: 28 Skipping: 29 Skipping: 30 Skipping: 31 Skipping: 32 Skipping: 33 Skipping: 34 Skipping: 35 Skipping: 36 Skipping: 37 Skipping: 38 Skipping: 39 Skipping: 40 Skipping: 41 Skipping: 42 Skipping: 43 Skipping: 44 Skipping: 45 Skipping: 46 Skipping: 47 Skipping: 48 Skipping: 49 Skipping: 50 Skipping: 51 Skipping: 52 Skipping: 53 Skipping: 54 Skipping: 55 Skipping: 56 Skipping: 57 Skipping: 58 Skipping: 59 Skipping: 60 Skipping: 61 Skipping: 62 Skipping: 63 Skipping: 64 Skipping: 65 Skipping: 66 Skipping: 67 Skipping: 68 Skipping: 69 Skipping: 70 Skipping: 71 Skipping: 72 Skipping: 73 Skipping: 74 Skipping: 75 Skipping: 76 Skipping:

## Batch

In [31]:
import os
import json
from pathlib import Path
from agents.batch_api_agents import prepare_batch_input, batch_api_agent
from openai import OpenAI
import importlib
import agents.batch_api_agents as batch_agents

# Reload the module to ensure the latest changes are loaded
importlib.reload(batch_agents)

llm_model = 'claude-3-5-sonnet-20240620'
project_root = '/Users/log/Github/textual_grounding/'
prompt_type = 'zero_shot_vanilla_cot'
few_shot_txt = None
sample_size = 2
json_datasets = ['logical_deduction_seven_objects','reasoning_about_colored_objects']
jsonl_datasets = ['GSM_Plus', 'MultiArith', 'SVAMP', 'p_GSM8K', 'StrategyQA', 'commonsenseQA','SPARTQA']
all_datasets = jsonl_datasets + json_datasets
all_datasets = ['MultiArith']
for dataset in all_datasets:
    if dataset in json_datasets:
        data_path = os.path.join(project_root, 'data', dataset, 'test.json')
    else:
        data_path = os.path.join(project_root, 'data', dataset, 'test.jsonl')
        
    save_dir = os.path.join(project_root, 'logan/results/final/VanillaCoT', dataset, f'{llm_model}')
    batch_dir = os.path.join(project_root, 'logan/batch_files/VanillaCoT', dataset, f'{llm_model}')
    os.makedirs(save_dir, exist_ok=True)  # Ensure the directory exists
    os.makedirs(batch_dir, exist_ok=True)  # Ensure the directory exists

    batch_results_path = os.path.join(save_dir, f'{prompt_type}_{few_shot_txt}_{dataset}_{llm_model}.jsonl')   
    batch_output_file = os.path.join(batch_dir, f'{prompt_type}_{few_shot_txt}_{dataset}_{llm_model}.jsonl')   

    ids, questions = load_data_size_specific(data_path, sample_size=sample_size)
    if few_shot_txt:
        with open(fewshot_prompt_path, 'r') as file:
            few_shot_prompt = file.read()
    else:
        few_shot_prompt = ""
    prompts = []
    for question in questions:
        prompt = get_prompt(prompt_type, few_shot_prompt, question)
        # print(prompt)
        prompts.append(prompt)

    # tasks = batch_agents.prepare_batch_input(
    #     llm_model=llm_model,
    #     ids=ids,
    #     prompts=prompts,
    #     batch_output_file=batch_output_file
    # )

    #Execute the batch processing with GPT-4 and save results
    batch_agents.batch_api_agent(
        llm_model=llm_model,
        ids=ids,
        prompts=prompts,
        batch_output_file=batch_output_file,
        batch_results_file=batch_results_path
    )


2024-11-10 17:46:09,175 - INFO - Batch input file created at: /Users/log/Github/textual_grounding/logan/batch_files/VanillaCoT/MultiArith/claude-3-5-sonnet-20240620/zero_shot_vanilla_cot_None_MultiArith_claude-3-5-sonnet-20240620.jsonl


[{'custom_id': '397', 'params': {'model': 'claude-3-5-sonnet-20240620', 'max_tokens': 1024, 'messages': [{'role': 'user', 'content': "Debby's class is going on a field trip to the zoo. If each van can hold 9 people and there are 40 students and 14 adults going, how many vans will they need?\n\nThink through your answer step by step. Put the concise form of your final answer in curly brackets e.g. {A}, {True} or {3.0}.\n"}]}}, {'custom_id': '433', 'params': {'model': 'claude-3-5-sonnet-20240620', 'max_tokens': 1024, 'messages': [{'role': 'user', 'content': 'John had 5 action figures, but needed 7 total for a complete collection. If each one costs $5, how much money would he need to finish his collection?\n\nThink through your answer step by step. Put the concise form of your final answer in curly brackets e.g. {A}, {True} or {3.0}.\n'}]}}]



2024-11-10 17:46:09,633 - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages/batches?beta=true "HTTP/1.1 200 OK"
2024-11-10 17:46:09,637 - INFO - Created batch with ID: msgbatch_014xdLrN6YFLTrqHmYTxghjf
2024-11-10 17:46:09,767 - INFO - HTTP Request: GET https://api.anthropic.com/v1/messages/batches/msgbatch_014xdLrN6YFLTrqHmYTxghjf?beta=true "HTTP/1.1 200 OK"


KeyboardInterrupt: 